In [ ]:
!git clone https://github.com/sunsmarterjie/yolov12.git
%cd yolov12
!wget https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl
%pip install -r requirements.txt
%pip install -e .

Cloning into 'yolov12'...
remote: Enumerating objects: 976, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 976 (delta 211), reused 193 (delta 192), pack-reused 753 (from 2)
Receiving objects: 100% (976/976), 1.61 MiB | 3.70 MiB/s, done.
Resolving deltas: 100% (465/465), done.
/content/yolov12
--2025-04-25 11:11:47--  https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/494232964/b7fb7022-424c-4ac2-b46b-a41f386edaa5?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250425%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250425T111147Z&X-Amz-Expires=300&X-A

Obtaining file:///content/yolov12
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.3.63-0.editable-py3-none-any.whl size=19689 sha256=a961cc43a49074bf1e26d1bd8ed94b9ea6f6b7f7450213b066654832733c5ecb
  Stored in directory: /tmp/pip-ephem-wheel-cache-8a1zswny/wheels/2f/97/19/dd82feac2203038bb82fbbfe2498f6943d715525dc3872e3e4
Successfully built ultralytics


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov12n.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/yolov12/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
FlashAttention is not available on this device. Using scaled_dot_product_attention instead.


100%|██████████| 5.26M/5.26M [00:00<00:00, 69.3MB/s]


In [ ]:
import kagglehub
fareselmenshawii_face_detection_dataset_path = kagglehub.dataset_download('fareselmenshawii/face-detection-dataset')

print(f'Data source import complete at {fareselmenshawii_face_detection_dataset_path}.')

Data source import complete at /kaggle/input/face-detection-dataset.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
# import tensorflow as tf

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# dataset_params = {
#     'data_dir':'/kaggle/input/face-detection-dataset',
#     'train_images_dir':'/kaggle/input/face-detection-dataset/images/train',
#     'train_labels_dir':'/kaggle/input/face-detection-dataset/labels/train',
#     'val_images_dir':'/kaggle/input/face-detection-dataset/images/val',
#     'val_labels_dir':'/kaggle/input/face-detection-dataset/labels/val',
#     'test_images_dir':'/kaggle/input/face-detection-dataset/images/val',
#     'test_labels_dir':'/kaggle/input/face-detection-dataset/labels/val',
#     'classes': ['face']
# }

In [ ]:
image_path = os.path.join(fareselmenshawii_face_detection_dataset_path, 'images', 'train', 'e3801c5cd028bd71.jpg')

# Now try to read the image
image = plt.imread(image_path)

plt.imshow(image) # Display the image
plt.show()

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow # Import cv2_imshow

# Define paths
image_path = os.path.join(fareselmenshawii_face_detection_dataset_path, 'images', 'train', 'e3801c5cd028bd71.jpg')
label_path = os.path.join(fareselmenshawii_face_detection_dataset_path, 'labels2', 'e3801c5cd028bd71.txt')

# Load image using OpenCV
image = cv2.imread(image_path)

# Read bounding box coordinates from the label file
with open(label_path, 'r') as f:
    for line in f:
        values = line.strip().split()
        x_min, y_min, x_max, y_max = map(float, values[2:])  # Extract coordinates

        # Convert float to integer for OpenCV
        x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)

        # Draw rectangle on the image
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green rectangle

        # Put label text
        cv2.putText(image, values[0], (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 255, 0), 2, cv2.LINE_AA)

# Display the image using cv2_imshow
cv2_imshow(image)

In [ ]:
result_of_original_yolo = model(image_path)
result_of_original_yolo[0].show()

In [ ]:
def convert_to_yolo_format(label_file, img_width, img_height):
    yolo_labels = []
    with open(label_file, 'r') as f:
        for line in f:
            values = line.strip().split()
            class_id = values[0]
            x_min, y_min, x_max, y_max = map(float, values[2:])

            # Calculate YOLO format
            x_center = (x_min + x_max) / 2 / img_width
            y_center = (y_min + y_max) / 2 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height

            yolo_labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_labels


In [ ]:
import os

# This should be where your actual dataset lives
dataset_path = "/kaggle/input/face-detection-dataset"

yaml_content = f"""\
path: {dataset_path}
train: images/train
val: images/val

names:
  0: face
"""

yaml_path = "/content/face.yaml"  # writable location

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"✅ Created face.yaml at: {yaml_path}")


In [ ]:
model.train(data="/content/face.yaml", epochs=2, imgsz=320,batch=32,cache = True)


In [ ]:
from ultralytics import YOLO

# Load model from the last checkpoint
model = YOLO("/content/yolov12/runs/detect/train/weights/last.pt")


In [ ]:
# import shutil

# # Copy to your main directory or give it a custom name
# shutil.copy(
#     "/content/yolov12/runs/detect/train4/weights/best.pt",
#     "/content/face_detector_yolov12.pt"
# )

# print("✅ Model saved as face_detector_yolov12.pt")


In [ ]:
# from google.colab import files
# files.download("/content/face_detector_yolov12.pt")

In [ ]:
image_path = "YoloTestImg.jpeg"
result_of_modified_yolo = model(image_path)
result_of_modified_yolo[0].show()